# Get news for instrument oil ("BZ=F")
https://levelup.gitconnected.com/how-to-import-json-from-s3-to-postgresql-on-rds-b132357af39e

In [15]:
# # Install required packages
# ! pip install yfinance --upgrade --no-cache-dir
# ! pip install psycopg2-binary
# ! pip install sqlalchemy

## Requirements & configuration

In [2]:
# Import required packages
# import os
import pandas as pd
import yfinance as yf
import json
from pprint import pprint
import psycopg2
from sqlalchemy import create_engine
from datetime import date

In [9]:
# DB configuration
config = {
    'host': 'datalake.cknmu1bvrxjg.us-east-1.rds.amazonaws.com',
    'port': '5432',
    'user': 'muser',
    'password': 'datalake',
    'dbname': 'datalake',
}

# Configure cnx_string for sqlalchemy
cnx_str = f'postgresql://{config["user"]}:{config["password"]}@{config["host"]}/{config["dbname"]}'

In [4]:
# Set ticker for required instrument "Brent Crude Oil"
brent = yf.Ticker("BZ=F")

## Getting news of instrument 'BZ=F'

In [19]:
# Get news and convert it into JSON
news = json.dumps(brent.news)
# pprint(news)

## Write news into datalake db

In [10]:
# Establish connection to database 'lakehouse'
try: 
    conn = psycopg2.connect(
        dbname=config['dbname'],
        user=config['user'],
        host=config['host'],
        password=config['password'],
        port=config['port']
    )

except psycopg2.Error as e: 
    print("Error: Could not make the connection to the postgres database")
    print(e)

# Create cursor
try: 
    cursor = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get the cursor to the database")
    print(e)

# Set auto commit feature
conn.set_session(autocommit=True)

# Create engine
engine = create_engine(cnx_str)

In [11]:
# Create table news_oil using the JSONB data type
sql = """
    CREATE TABLE IF NOT EXISTS news_oil (
      ingested_at timestamp DEFAULT CURRENT_TIMESTAMP,
      news jsonb NOT NULL
      );
"""
cursor.execute(sql)

In [18]:
# Upload news into db
sql = """
    INSERT INTO news_oil (news)
    VALUES (%s);
"""
cursor.execute(sql, [news])

## Close connection

In [13]:
# Close connection
cursor.close()
conn.close()